# 🚀 AGI Çekirdek Avcısı - Google Colab

Bu notebook, AGI Core Hunter deneylerini Google Colab'da çalıştırmanızı sağlar.

**Avantajlar:**
- ✅ Ücretsiz GPU (T4)
- ✅ 12 saat sürekli çalışma
- ✅ Hızlı kurulum
- ✅ Wandb entegrasyonu

**Kullanım:** Runtime → Change runtime type → GPU

In [ ]:
# 🔧 Kurulum ve Bağımlılıklar
!nvidia-smi  # GPU kontrolü

# Seçenek 1: GitHub'dan klonla (eğer repo hazırsa)
# !git clone https://github.com/[username]/agi_core_hunter.git
# %cd agi_core_hunter

# Seçenek 2: Manuel dosya yükleme
print('📁 Dosyaları yüklemek için:')
print('1. Sol panelden Files ikonuna tıkla')
print('2. Upload butonuna tıkla')
print('3. Tüm proje dosyalarını sürükle-bırak')
print('4. Veya zip dosyası yükle ve aşağıdaki kodu çalıştır:')
print()
print('from google.colab import files')
print('uploaded = files.upload()')
print('!unzip agi_core_hunter.zip')
print('%cd agi_core_hunter')

# Bağımlılıkları yükle (Colab için optimize)
!pip install jax[cuda] flax optax chex wandb tqdm matplotlib seaborn -q

# Test kurulumu
!python test_setup.py

In [ ]:
# 📊 Wandb Kurulumu
import wandb

# Wandb login (API key gerekli)
wandb.login()

# Veya offline mode
# import os
# os.environ['WANDB_MODE'] = 'offline'

In [ ]:
# 🎮 Hızlı Demo (2 dakika)
!python experiments/01_mdl_vs_ood/demo.py

In [ ]:
# ⚡ Hızlı Deney (500 episode, ~5 dakika)
%cd experiments/01_mdl_vs_ood

# Manifest'i düzenle (daha az episode)
import json

with open('manifest.json', 'r') as f:
    config = json.load(f)

# Hızlı test için parametreleri azalt
config['training']['total_episodes'] = 500
config['training']['eval_frequency'] = 100
config['evaluation']['ood_tests'][0]['episodes'] = 50

with open('manifest_fast.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Hızlı test konfigürasyonu hazır!")

In [ ]:
# 🧪 Deneyi Çalıştır
!python train.py --manifest manifest_fast.json

In [ ]:
# 📊 Sonuçları Analiz Et
import json
import matplotlib.pyplot as plt
import pandas as pd

# Sonuçları yükle
with open('results.json', 'r') as f:
    results = json.load(f)

# Hızlı görselleştirme
agents = list(results.keys())
betas = [results[agent]['agent_config']['beta'] for agent in agents]
ood_ratios = []

for agent in agents:
    ratios = [v for k, v in results[agent]['ood_performance_ratios'].items() 
             if k.endswith('_performance_ratio')]
    ood_ratios.append(sum(ratios) / len(ratios) if ratios else 0)

plt.figure(figsize=(10, 6))
plt.scatter(betas, ood_ratios, s=100)
for i, agent in enumerate(agents):
    plt.annotate(agent, (betas[i], ood_ratios[i]))

plt.xlabel('Beta (Compression Pressure)')
plt.ylabel('Average OOD Performance Ratio')
plt.title('MDL Principle Test Results')
plt.grid(True, alpha=0.3)
plt.show()

# Sonuç özeti
print("\n🎯 SONUÇ ÖZETİ:")
for i, agent in enumerate(agents):
    print(f"{agent}: β={betas[i]}, OOD Ratio={ood_ratios[i]:.3f}")

# Hipotez testi
import numpy as np
correlation = np.corrcoef(betas, ood_ratios)[0, 1]
print(f"\n📈 Beta-OOD Korelasyonu: {correlation:.3f}")

if correlation > 0.3:
    print("✅ HİPOTEZ DESTEKLENDİ: Yüksek β daha iyi OOD performansı sağlıyor!")
elif correlation < -0.3:
    print("❌ HİPOTEZ REDDEDİLDİ: Yüksek β OOD performansını kötüleştiriyor.")
else:
    print("❓ HİPOTEZ BELİRSİZ: Daha fazla veri gerekli.")

In [ ]:
# 🔥 TAM DENEY (5000 episode, ~30 dakika)
# Sadece sonuçlar tatmin edici değilse çalıştır

# !python train.py  # Orijinal manifest ile

In [ ]:
# 💾 Sonuçları Kaydet
from google.colab import files
import shutil

# Sonuçları zip'le
!zip -r experiment_results.zip results.json *.png wandb/

# İndir
files.download('experiment_results.zip')

print("✅ Sonuçlar indirildi!")

## 🚀 Sonraki Adımlar

1. **Sonuçları Analiz Et:** İndirilen dosyaları yerel makinende incele
2. **Blog Yazısı Yaz:** Bulgularını paylaş
3. **Sonraki Deneyi Planla:** Farklı latent boyutları test et
4. **Toplulukla Paylaş:** GitHub'da sonuçları güncelle

## 💡 İpuçları

- **GPU Süresi:** Colab Pro ile 24 saat
- **Veri Kaybı:** Sonuçları düzenli olarak indir
- **Hızlandırma:** JAX compilation cache kullan
- **Monitoring:** Wandb dashboard'unu takip et